In [7]:
import pandas as pd
import math

# Inputs
annual_demand_kWh = 10000
input_profiles = pd.read_csv('profiles.csv')
pv_to_load_ratio = 2
hybrid_system = False
peak_demand_backup_hrs = 4

# Constants
battery_min_c_rate = 5
grid_export_rating = 1000   #kW ac
grid_import_rating = 1000   #kW ac
soc_low = 0.3          #%
soc_start = 0.5        #%
# Efficiencies
e_pv = 0.98
e_inv = 0.95
e_gc = 0.95
e_bch = 0.95
e_bdc = 0.95

# Scale load profile
load_profile_input = input_profiles['load'].tolist()
load_scalar = annual_demand_kWh / sum(load_profile_input)
load_profile = [i* load_scalar for i in load_profile_input]

# Scale solar profile
solar_target_generation = annual_demand_kWh * pv_to_load_ratio
solar_profile_input = input_profiles['solar'].tolist()
solar_scalar = solar_target_generation / sum(solar_profile_input)
solar_profile = [i* solar_scalar for i in solar_profile_input]

# Extract grid profile
grid_profile = input_profiles['grid'].tolist()

# Size battery
peak_demand = max(load_profile)
batt_capacity = math.ceil(peak_demand * peak_demand_backup_hrs)
batt_ch_rating = math.ceil(batt_capacity/battery_min_c_rate)
batt_dc_rating = -batt_ch_rating

# Size inverter & grid charger
grid_ch_rating = batt_ch_rating
inv_rating = max(max(solar_profile),max(load_profile)) * 1.1




In [8]:
import simulation_models
import pandas as pd


les = simulation_models.PVBattBasic(
    time_inc = 1,           #hrs

    # Profiles
    solar_profile = solar_profile,
    load_profile = load_profile,
    grid_profile = grid_profile,

    # System specs
    hybrid_system = hybrid_system,   #T/F, False: off-grid
    batt_ch_rating = batt_ch_rating,     #kW dc
    batt_dc_rating = batt_dc_rating,   #kW dc
    inv_rating = inv_rating,       #kW ac, output
    grid_ch_rating = grid_ch_rating,    #kW dc, output
    soc_low = soc_low,          #%
    soc_start = soc_start,        #%
    batt_capacity = batt_capacity,    #kWh

    # Grid connection
    grid_export_rating = grid_export_rating,   #kW ac
    grid_import_rating = grid_import_rating,   #kW ac

    # Efficiency
    e_pv = e_pv,
    e_inv = e_inv,
    e_gc = e_gc,
    e_bch = e_bch,
    e_bdc = e_bdc,
)

les.simulate()



In [9]:
# Plot day profile
import datetime

datetime_from = datetime.datetime(2023,7,1,0,0,0)
datetime_to = datetime.datetime(2023,7,1,23,0,0)

fig1 = les.graph_load(les.output_df, datetime_from=datetime_from, datetime_to=datetime_to)
fig1.show()

fig2 = les.graph_solar(les.output_df, datetime_from=datetime_from, datetime_to=datetime_to)
fig2.show()

fig3 = les.graph_batt(les.output_df, datetime_from=datetime_from, datetime_to=datetime_to)
fig3.show()

fig4 = les.graph_grid(les.output_df, datetime_from=datetime_from, datetime_to=datetime_to)
fig4.show()


In [10]:
# Summarise monthly data
monthly_data = les.output_df.groupby(pd.Grouper(key='datetime', freq='M')).sum()
monthly_data['datetime'] = monthly_data.index

In [11]:

datetime_from = datetime.datetime(2023,1,1,0,0,0)
datetime_to = datetime.datetime(2023,12,31,23,0,0)

fig1 = les.graph_load(monthly_data, datetime_from=datetime_from, datetime_to=datetime_to)
fig1.show()

fig2 = les.graph_solar(monthly_data, datetime_from=datetime_from, datetime_to=datetime_to)
fig2.show()

fig3 = les.graph_grid(monthly_data, datetime_from=datetime_from, datetime_to=datetime_to)
fig3.show()

In [12]:
import simulation_models
monthly_grid_consumption = les.calc_monthly_grid_consumption()
monthly_grid_demand = les.calc_monthly_grid_demand()

cost = simulation_models.CostBasic(
    # Financial parameters
    project_years = 25,   #Total number of years for the project
    discount_rate = 0.07, #%

    # Cost inputs
    cost_pv = 27000,    #INR/kWpk
    cost_batt = 7000,   #INR/kWh
    cost_inv = 15000,   #INR/kWpk

    # Markup inputs
    markup_bos = 0.1,       #% markup on CAPEX above
    markup_install = 0.05,   #% markup on CAPEX above, inc. other markup above
    markup_transport = 0.05, #% markup on CAPEX above, inc. other markup above
    markup_margin = 0.15,    #% markup on CAPEX above, inc. other markup above

    # System details
    #dict{'pv_kW': 10, 'batt_kWh':100, ...}
    system_specs = {
        'pv_kW': 10,
        'batt_kWh': 100,
        'inv_kW': 10,
    },
    #list of length 12 with total demand/import {'import':10000, 'export:3000'}
    monthly_grid_consumption = monthly_grid_consumption,
    monthly_grid_demand = monthly_grid_demand,

    # OPEX inputs
    opex_percentage = 0.05, #% of total CAPEX
    opex_inf = 0.05, #% annual increase

    # Grid cost inputs
    net_metering = True,
    demand_import_slabs = [
        [50, 120],   #[peak kW demand, Rs/kW cost]
        [100, 150],
        [100000000, 175],
    ],
    demand_export_slabs = [
        [100000000, 50],   #[peak kW demand, Rs/kW cost]
    ],

    energy_import_slabs = [
        [200, 7.35], #[monthly kWh consumption, Rs/kWh cost]
        [400, 8],
        [100000000, 8.6],
    ],

    energy_export_slabs = [
        [100000000, -2], #[monthly kWh consumption, Rs/kWh cost]
    ],

    # REPEX inputs
    #list['pv': {rep_yr: 25, rep_inf: 1.2}, 'batt': {rep_yr: 25, rep_inf: 1.2},...]
    repex_inputs = [
        # component, replacement year, replacement inflation
        ['pv', 26, 1.2],
        ['batt', 15, 1.2],
        ['inv', 15, 1.2],
    ],
)

cost.calc_capex()
cost.calc_opex()
cost.calc_repex()
cost.calc_grid_costs()
cost.calc_npv()

print(cost.npv)
print(cost.cashflow)



3570411.7250799662
[1120000, 144190.0, 146990.0, 149930.0, 153017.0, 156258.35, 159661.76750000002, 163235.355875, 166987.62366875002, 170927.50485218753, 175064.3800947969, 179408.09909953678, 183969.00405451359, 188757.9542572393, 193786.35197010124, 1219066.1695686064, 204609.97804703662, 210430.97694938848, 216543.0257968579, 222960.6770867008, 229699.21094103585, 236774.67148808765, 244203.90506249203, 252004.60031561664, 260195.33033139748, 268795.59684796736]
